In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class zeroCNN(nn.Module):
    def __init__(self, num_classes):
        super(zeroCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 28 * 28)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
    
num_classes = 10
model = zeroCNN(num_classes)
#model.load_state_dict(torch.load('/kaggle/working/custom.pth'))

n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of Params: {n_parameters / 1000000:.1f}M")

Number of Params: 51.5M


In [3]:
torch.save(model.state_dict(), 'custom.pth')

In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models

def custom_tensorflow(num_classes):
    inputs = tf.keras.Input(shape=(224, 224, 3))
    
    x = layers.Conv2D(32, (3, 3), padding='same', activation='relu')(inputs)  # Layer 1
    
    x = layers.MaxPooling2D((2, 2))(x)  # Layer 2
    
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)  # Layer 3
    
    x = layers.MaxPooling2D((2, 2))(x)  # Layer 4
    
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)  # Layer 5
    
    x = layers.MaxPooling2D((2, 2))(x)  # Layer 6
    
    
    x = layers.Flatten()(x)  # Layer 10
    x = layers.Dropout(0.5)(x)  # Layer 11
    x = layers.Dense(512, activation='relu')(x)  # Layer 12
    x = layers.Dropout(0.5)(x)  # Layer 13
    outputs = layers.Dense(num_classes, activation='softmax')(x)  # Layer 14
    
    model = models.Model(inputs, outputs)
    return model

# Create an instance of the model
num_classes = 10
tf_model = custom_tensorflow(num_classes)

n_parameters = tf_model.count_params()
print(f"Number of Params: {n_parameters / 1000000:.1f}M")

Number of Params: 51.5M


In [9]:
pytorch_model = zeroCNN(num_classes)

pytorch_model.load_state_dict(torch.load('/kaggle/working/custom.pth', map_location=torch.device('cpu')))
pytorch_model.eval()

# Helper function to convert PyTorch tensors to NumPy arrays
def pt_to_np(tensor):
    return tensor.detach().cpu().numpy()

In [11]:
# Map PyTorch weights to TensorFlow model
tf_model.layers[1].set_weights([pt_to_np(pytorch_model.conv1.weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.conv1.bias)])


tf_model.layers[3].set_weights([pt_to_np(pytorch_model.conv2.weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.conv2.bias)])


tf_model.layers[5].set_weights([pt_to_np(pytorch_model.conv3.weight.permute(2, 3, 1, 0)), pt_to_np(pytorch_model.conv3.bias)])



tf_model.layers[9].set_weights([pt_to_np(pytorch_model.fc1.weight.T), pt_to_np(pytorch_model.fc1.bias)])
tf_model.layers[11].set_weights([pt_to_np(pytorch_model.fc2.weight.T), pt_to_np(pytorch_model.fc2.bias)])

In [12]:
tf_model.save('custom.h5')

In [13]:
load_tf_model = models.load_model('/kaggle/working/custom.h5')

In [14]:
n_parameters = load_tf_model.count_params()
print(f"Number of Params: {n_parameters / 1000000:.1f}M")

Number of Params: 51.5M
